## 경매물건정보 크롤링
  - 일자별 종결된 경매물건 list
  - 건물 등기 사항(권리분석용)
  
### 크롤링 사이트
http://www.speedauction.co.kr/
 - 크롤링 참고사이트: https://book.coalastudy.com/data-crawling/week-6/stage-2
 
### 방법
 1. 로그인 -> 상단메뉴 '경매검색' 페이지 이동
 2. '법원별경매' -> 서울 법원리스트 탐색
 3. 용도검색 '상업용' / 매각기일 2018.7.1 ~ 2018.12.31 / 150개씩 선택 검색
 4. 하단 물건정보 리스트 및 상세조회 팝업창 크롤링

In [447]:
import bs4
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd

import warnings
warnings.filterwarnings('ignore')           # 성가신 warning은 off 해준다. 

import os
print("아래 폴더에 chromedriver 파일 있어야 해요~")
os.getcwd()  # 해당폴더에 chromedriver 파일 있어야함.

아래 폴더에 chromedriver 파일 있어야 해요~


'C:\\Users\\student\\Desktop\\Ant_Project\\경매프로젝트\\크롤링 소스코드'

## 함수 선언

In [448]:
# 로그인 
def login(login_id="", login_pw=""):
    # 접속할 페이지의 주소
    url = 'http://www.speedauction.co.kr/v3/M_etc/customercenter/members.htm'
    # 크롬브라우져 해당 페이지 접속
    driver.get(url)
    
    # 로그인 id/pw 입력
    if login_id == "":
        login_id = input('아이디 입력: ')
    if login_pw == "":    
        login_pw = input('패스워드 입력: ')
    
    driver.refresh() # 새로고침
    input_id = driver.find_element_by_name('id')
    input_id.send_keys(login_id)

    input_pw = driver.find_element_by_name('pw')
    input_pw.send_keys(login_pw)

    # 로그인 버튼 클릭
    # TAB = '\ue004'
    # input_pw.send_keys(Keys.TAB)  # Tab키를 통해 로그인버튼으로 이동
    login_btn = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > table > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > input[type="image"]')
    login_btn.click()
    
# 검색 기간
def inputSearchPeriod(startYmd="", endYmd=""):    
    if startYmd == "":
        startYmd = input('검색일자(시작일자YYYYmmdd): ')
        endYmd = input('검색일자(종료일자YYYYmmdd): ')
    
    input_startYmd = driver.find_element_by_css_selector('#sell_yyyymmdd_ss')
    input_startYmd.clear()
    input_startYmd.send_keys(startYmd)
    input_endYmd = driver.find_element_by_css_selector('#sell_yyyymmdd_ee')
    input_endYmd.clear()
    input_endYmd.send_keys(endYmd)

## 로그인 (id/pw 입력)

In [449]:
#웹 드라이버 가동
driver = webdriver.Chrome('chromedriver')

In [450]:
# 로그인 실행
login('gogo4845', 'wjsxodJT48!')

# 팝업창 닫기
driver.switch_to_window(driver.window_handles[-1])
driver.close()
# 활성창 변경
driver.switch_to_window(driver.window_handles[-1])

## 경매검색 페이지 접속
 - 검색 상세조건 설정

In [451]:
# 활성창 확인
driver.switch_to_window(driver.window_handles[-1])
print( driver.current_url )

# 경매 검색 메뉴 클릭
search_tag = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(1) > td:nth-child(1)')
search_tag.click()

# 상업용 체크박스 클릭
commer_checkbox_tag = driver.find_element_by_css_selector('#trYongdo > td > table > tbody > tr:nth-child(5) > td:nth-child(1) > font > strong > input[type="checkbox"]')
commer_checkbox_tag.click()

# 서울지법 전체 선택
input_court = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td:nth-child(2) > select:nth-child(1)')
court_select_tag = Select(input_court)
court_select_tag.select_by_value('A00')

# 검색일자 입력
# 매각기일 2018.7.1 ~ 2018.12.31
inputSearchPeriod('20180701', '20181231')

# 검색
search = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(3) > td > img')
search.click()

# 한페이지 목록갯수 설정 25/50/100/150개씩
list_cnt_per_page = 100
num_of_list = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(12) > td > table > tbody > tr > td:nth-child(3) > select')
num_of_list_tag = Select(num_of_list)
num_of_list_tag.select_by_value(f'{list_cnt_per_page}')

# 용도검색 '상업용' combo 재 선택
# 0:전체
# H:주거용
# S:상업용
# L:토지
# E:기타부동산
# C:차량외
use_combo = driver.find_element_by_css_selector('#trSrc > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(5) > td:nth-child(2) > select')
select_combo = Select(use_combo)
select_combo.select_by_value('S')

http://www.speedauction.co.kr/v3/?maintab=1&lefttab=2&setno=&chgcnt=0&salemoney=


In [452]:
# 조회건수 및 검색기간 확인
chk_value = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(6) > td > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1)')
# print( chk_value.text.split(' ') )
total_cnt = chk_value.text.split(' ')[3].replace("건", "")
period = chk_value.text.split(' ')[4].replace("(", "").replace(")", "")
total_cnt = int(total_cnt)
print('기간:', period )
print('조회건수:', total_cnt)

# 페이지 수 계산 ( 조회건수 / 페이지당 목록수 )
total_page_cnt = total_cnt / list_cnt_per_page
if total_cnt % list_cnt_per_page > 0:
        total_page_cnt += 1
total_page_cnt = int(total_page_cnt)
print('페이지수:', total_page_cnt )

기간: 2018.07.01~2018.12.31
조회건수: 332
페이지수: 4


In [ ]:
# 파일 최초 저장여부
chk = False

# 상세화면 로딩 시간이 걸리므로 딜레이 줌.
delay_time = 10

# 페이지수 만큼 반복 total_page_cnt
for page in range(total_page_cnt):
    
    # 수집항목 정의
    data_dic = {
        'Auction_class' : [],     #경매구분
        'Date_of_sale' : [],      #매각기일
        'Address' : [],           #주소
        'Address_loadname' : [],  #도로명주소
        'Post_number' : [],       #우편번호
        'Usage' : [],             #용도
        #'Usage_on_paper' : [],    #대장용도
        'Land_area' : [],         #대지권
        'Exclusive_area' : [],    #전용면적
        'For_sale' : [],          #매각대상
        'Date_of_reciep' : [],    #사건접수날짜
        'Price' : [],             #감정가
        'Lowest_price' : [],      #최저가
        'Billing_amount' : [],    #청구금액
        'Release_Date' : [],      #개시결정
        'Auction_Status' : [],    #기일현황
        'Building_Status' : [],   #물건현황,토지이용계획
        'Evaluation_Status' : [], #감정평가 현황
        'Tenant_status' : [],     #임차인/대항력여부
        'Registration' :[],       #등기사항,소멸여부
        'detail_link' :[]         #상세페이지 링크
    }
    
    print('현재 페이지:', page+1 )
    
    # 2페이지부터 클릭
    if page > 0:
        # 페이지 클릭
        page_idx = page*2 + 1
        page_btn = driver.find_element_by_css_selector(f'body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(16) > td > a:nth-child({page_idx})')
        page_btn.click()

    # 페이지당 목록수 만큼 반복
    if total_cnt < list_cnt_per_page:
        loop_cnt_per_page = total_cnt
    else:
        loop_cnt_per_page = list_cnt_per_page
    loop_cnt_per_page = int(loop_cnt_per_page)    
    print('페이지당 목록수:', loop_cnt_per_page)
    
    for idx in range(loop_cnt_per_page):
        list_idx = idx+3
        
        # 물건 클릭
        incident = driver.find_element_by_css_selector(f'body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(13) > td > table > tbody > tr:nth-child({list_idx}) > td:nth-child(4)')
        incident.click()
        # 상세페이지 활성창 변경
        driver.switch_to_window(driver.window_handles[-1])
        # 상세페이지 링크
        detail_link_str = driver.current_url
        
        # 화면 로딩 시간이 걸리므로 딜레이 줌.
        time.sleep(delay_time)
        
        soup = bs4.BeautifulSoup(driver.page_source)
        # print( soup.prettify() )
        strong_tag = soup.find_all('strong')
        # '정정공고','변경공고' 항목에 따라 tr태그 구조가 바뀜
        jj_none = True  # 항목 여부 (없으면 skip~)
        for jj_tag in strong_tag:
        #     print( jj_tag.text.strip() )
            if jj_tag.text.strip() == "정정공고":
                jj_none = False
            elif jj_tag.text.strip() == "변경공고":
                jj_none = False
                
        # select 태그가 있으면  '물번' 항목여부
        # select_tag = False
        # tag = soup.find_all('select')
        # if len(tag) > 0:
        #     select_tag = True
        
        # 화면 로딩 시간이 걸리므로 딜레이 줌.
        # time.sleep(delay_time)
        
        # 항목별 tag 가져오기
        Auction_class_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(7) > td > table > tbody > tr > td:nth-child(1) > font > strong')
        # print( Auction_class_tag.text )
        Date_of_sale_tag = driver.find_element_by_css_selector('#tdbx00 > table > tbody > tr > td:nth-child(2) > b')
        # print( Date_of_sale_tag.text )
        Address_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(1) > td:nth-child(2) > table > tbody > tr > td > font > b')
        # print( Address_tag.text )        
        Address_loadname_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(1) > td:nth-child(2) > table > tbody > tr > td')
        # print( Address_loadname_tag.text )
        Post_number_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(1) > td:nth-child(2) > table > tbody > tr > td > font')
        # print( Post_number_tag.text )
        Usage_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(2) > td:nth-child(2)')
        # print( Usage_tag.text )
        # Usage_on_paper_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(3) > td:nth-child(2)')
        # print( Usage_on_paper_tag.text )
        Land_area_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(3) > td:nth-child(2)')
        # print( Land_area_tag.text )
        Exclusive_area_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(4) > td:nth-child(2)')
        For_sale_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(5) > td:nth-child(4)')
        Date_of_reciep_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(5) > td:nth-child(2)')
        Price_tag = driver.find_element_by_css_selector('#tdbx01')
        Lowest_price_tag = driver.find_element_by_css_selector('#tdbx02')
        Billing_amount_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(5) > td:nth-child(6)')
        Release_Date_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(8) > td > table > tbody > tr:nth-child(6) > td:nth-child(6)')
        Auction_Status_tag = driver.find_element_by_css_selector('#tbl_glist')
        # print( Auction_Status_tag.text )

        # print( jj_none )
        # '정정공고' '변경공고' 항목이 없으면
        if jj_none == True:
            Building_Status_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(17) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr:nth-child(1) > td > table > tbody')
            # print( Building_Status_tag.text )
            Evaluation_Status_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(17) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table')        
            # print( Evaluation_Status_tag.text )    
            Tenant_status_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(17) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(3)')
            # print( Tenant_status_tag.text )
            Registration_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(17) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(4)')
            # print( Registration_tag.text )
        # 항목이 있으면
        else:
            Building_Status_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(18) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr:nth-child(1) > td > table')
            Evaluation_Status_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(18) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody')
            Tenant_status_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(18) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(3) > table > tbody')
            Registration_tag = driver.find_element_by_css_selector('#printArea > tbody > tr > td > table > tbody > tr:nth-child(18) > td > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td:nth-child(4) > table > tbody') 
        
        # 딕셔너리에 담기
        data_dic['Auction_class'].append(Auction_class_tag.text.split('(')[1].split(')')[0])
        data_dic['Date_of_sale'].append(Date_of_sale_tag.text.split(' ')[0])
        data_dic['Address'].append(Address_tag.text.strip())
        data_dic['Address_loadname'].append(Address_loadname_tag.text.strip())
        data_dic['Post_number'].append(Post_number_tag.text.split(')')[0].split('(')[1])
        data_dic['Usage'].append(Usage_tag.text.strip())
        # data_dic['Usage_on_paper'].append(Usage_on_paper_tag.text.strip())
        data_dic['Land_area'].append(Land_area_tag.text.strip())
        data_dic['Exclusive_area'].append(Exclusive_area_tag.text.strip())
        data_dic['For_sale'].append(For_sale_tag.text.strip())
        data_dic['Date_of_reciep'].append(Date_of_reciep_tag.text.strip())
        data_dic['Price'].append(Price_tag.text)
        data_dic['Lowest_price'].append(Lowest_price_tag.text.strip())
        data_dic['Billing_amount'].append(Billing_amount_tag.text.strip())
        data_dic['Release_Date'].append(Release_Date_tag.text.strip())
        data_dic['Auction_Status'].append(Auction_Status_tag.text.strip())
        data_dic['Building_Status'].append(Building_Status_tag.text.strip())
        data_dic['Evaluation_Status'].append(Evaluation_Status_tag.text.strip())
        data_dic['Tenant_status'].append(Tenant_status_tag.text.strip())
        data_dic['Registration'].append(Registration_tag.text.strip())
        data_dic['detail_link'].append(detail_link_str.strip())
        # print( data_dic )
        
        print(page+1, '페이지 ', idx+1,'/',loop_cnt_per_page, '수집 완료' )
        # 상세페이지 크롤링 완료후 닫기
        driver.close()
        # 경매물건 list로 활성창 변경
        driver.switch_to_window(driver.window_handles[-1])
    
    # 페이지별 데이터프레임 저장
    df = pd.DataFrame.from_dict(data_dic)

    title = '상가경매물건(' + period + ').csv'

    if chk==False:
        chk = True
        df.columns = ['경매구분', '매각기일','주소','도로명주소','우편번호','용도','대지권','전용면적','매각대상','사건접수날짜','감정가','최저가','청구금액','개시결정','기일현황','물건현황_토지이용계획','감정평가현황','임차인_대항력여부','등기사항_소멸여부','상세페이지링크']
        df.to_csv(title, index=False, encoding='utf-8-sig')
    else:
        df.to_csv(title, index=False, encoding='utf-8-sig', mode='a', header=False )
    print(page+1, '페이지 엑셀저장 완료')
          
print("수집 완료~!!!")        

현재 페이지: 1
페이지당 목록수: 100
